In [1]:
%matplotlib inline

import matplotlib
matplotlib.rcParams['figure.figsize'] = (6, 6)

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

import ipywidgets
from ipywidgets import interact

import sys
sys.path.append("../")

import datapipe
from datapipe.io.images import image_files_in_dir
from datapipe.denoising.wavelets_mrtransform import wavelet_transform

In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;
// see https://github.com/ipython/ipython/issues/2172

<IPython.core.display.Javascript object>

## Display input fits files

In [3]:
#DIR_PATH = "/Users/jdecock/astri_data/fits_cropped/proton/"
#DIR_PATH = "Users/jdecock/astri_data/fits/proton/"
#DIR_PATH = "/Users/jdecock/astri_data/fits/gamma/"
#DIR_PATH = "/Users/jdecock/data/gct/fits/proton/"
#DIR_PATH = "/Users/jdecock/astri_data/fits_flashcam/gamma/"
#DIR_PATH = "/Users/jdecock/astri_data/fits_flashcam/proton/"
#DIR_PATH = "/Volumes/ramdisk/flashcam/fits/gamma/"
#DIR_PATH = "/Users/jdecock/data/grid_prod3b_north/fits/lst/gamma/"
DIR_PATH = "/Users/jdecock/data/sst1m_mini_array_konrad/fits/sst1m/gamma/"

file_path_list = list(image_files_in_dir(DIR_PATH, max_num_files=5))
file_path_list

['/Users/jdecock/data/sst1m_mini_array_konrad/fits/sst1m/gamma/gamma_20deg_180deg_run2203___cta-prod3-sst-dc_desert-2150m-Paranal-sst-dc.simtel.gz_TEL001_EV00109.fits',
 '/Users/jdecock/data/sst1m_mini_array_konrad/fits/sst1m/gamma/gamma_20deg_180deg_run2203___cta-prod3-sst-dc_desert-2150m-Paranal-sst-dc.simtel.gz_TEL001_EV02301.fits',
 '/Users/jdecock/data/sst1m_mini_array_konrad/fits/sst1m/gamma/gamma_20deg_180deg_run2203___cta-prod3-sst-dc_desert-2150m-Paranal-sst-dc.simtel.gz_TEL001_EV02308.fits',
 '/Users/jdecock/data/sst1m_mini_array_konrad/fits/sst1m/gamma/gamma_20deg_180deg_run2203___cta-prod3-sst-dc_desert-2150m-Paranal-sst-dc.simtel.gz_TEL001_EV02607.fits',
 '/Users/jdecock/data/sst1m_mini_array_konrad/fits/sst1m/gamma/gamma_20deg_180deg_run2203___cta-prod3-sst-dc_desert-2150m-Paranal-sst-dc.simtel.gz_TEL001_EV02802.fits']

In [4]:
def plot(data, title="", log=False):
    fig = plt.figure()
    ax1 = fig.add_subplot(111)

    ax1.set_title(title)
    
    if log:
        # See http://matplotlib.org/examples/pylab_examples/pcolor_log.html
        #     http://stackoverflow.com/questions/2546475/how-can-i-draw-a-log-normalized-imshow-plot-with-a-colorbar-representing-the-raw
        #im = ax1.pcolor(x, y, image_array, norm=LogNorm(vmin=0.01, vmax=image_array.max()), cmap=self.color_map)  # TODO: "vmin=0.01" is an arbitrary choice...
        im = ax1.imshow(data, interpolation='nearest', origin='lower', norm=LogNorm(vmin=0.01, vmax=data.max()), cmap="gnuplot2")   # cmap=cm.inferno and cmap="inferno" are both valid
    else:
        im = ax1.imshow(data, interpolation='nearest', origin='lower', cmap="gnuplot2")   # cmap=cm.inferno and cmap="inferno" are both valid
        #im = ax1.pcolor(x, y, image_array, cmap=self.color_map, vmin=z_min, vmax=z_max)

    plt.colorbar(im, ax=ax1)  # draw the colorbar
    
    #ax2 = fig.add_subplot(212)
    #ax2.hist(data)
    
    plt.show()

In [8]:
@interact(file_path=file_path_list)
def gui(file_path):
    images, metadata = datapipe.io.images.load_benchmark_images(file_path)

    print(metadata["npe"])
    
    for key, data in images.items():
        print(key, data.ndim, data.shape)
        if data.ndim == 2:
            plot(data, title=key)
        elif data.ndim == 3:
            for channel in range(data.shape[0]):
                plot(data[channel], title=key + " [{}]".format(channel))
            
    for key, val in metadata.items():
        print("{}: {}".format(key, val))
        
    print(images["reference_image"])
    print(images["input_image"])
    
    plan_imgs = wavelet_transform(images["input_image"])
    for plan_index, plan_img in enumerate(plan_imgs):
        plot(plan_img, title="Input image - WT plan {}".format(plan_index))
        
    plan_imgs = wavelet_transform(images["reference_image"])
    for plan_index, plan_img in enumerate(plan_imgs):
        plot(plan_img, title="PE image - WT plan {}".format(plan_index))